In [2]:
import os
from glob import glob
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
def read_image(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img_rgb

In [4]:
data_dir = '../影像資料生成競賽/34_Competition 1_Training dataset/Training dataset/img'
gt_dir = '../影像資料生成競賽/34_Competition 1_Training dataset/Training dataset/label_img'

river_data_paths = glob(os.path.join(data_dir, 'TRA_RI_*.jpg'))
road_data_paths = glob(os.path.join(data_dir, 'TRA_RO_*.jpg'))

In [5]:
data_df = pd.DataFrame(
    {
        'data_path': river_data_paths + road_data_paths,
        'label': ['river'] * len(river_data_paths) + ['road'] * len(road_data_paths)
    }
)
data_df

,data_path,label
0,../影像資料生成競賽/34_Competition 1_Training dataset/...,river
1,../影像資料生成競賽/34_Competition 1_Training dataset/...,river
2,../影像資料生成競賽/34_Competition 1_Training dataset/...,river
3,../影像資料生成競賽/34_Competition 1_Training dataset/...,river
4,../影像資料生成競賽/34_Competition 1_Training dataset/...,river
...,...,...
4315,../影像資料生成競賽/34_Competition 1_Training dataset/...,road
4316,../影像資料生成競賽/34_Competition 1_Training dataset/...,road
4317,../影像資料生成競賽/34_Competition 1_Training dataset/...,road
4318,../影像資料生成競賽/34_Competition 1_Training dataset/...,road


In [6]:
data_df['gt_data_path'] = data_df['data_path'].apply(
    lambda p: os.path.join(gt_dir, os.path.basename(p)).replace('.jpg', '.png')
)

In [7]:
import numpy as np
from PIL import Image
from collections import deque

def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    return np.array(image)

def mark_paths(image, label):
    rows, cols, _ = image.shape
    visited = np.zeros((rows, cols), dtype=bool)
    
    def is_white(pixel):
        return all(pixel == [255, 255, 255])

    def bfs(x, y):
        queue = deque([(x, y)])
        while queue:
            x, y = queue.popleft()
            # print(f'{x = }, {y = }')
            if visited[x, y] or is_white(image[x, y]):
                continue
            visited[x, y] = True
            image[x, y] = [255, 0, 0]  # Mark as red  
            if x > 0: queue.append((x - 1, y))
            if x < rows - 1: queue.append((x + 1, y))
            if y > 0: queue.append((x, y - 1))
            if y < cols - 1: queue.append((x, y + 1))

    # Iterate over the boundary
    for x in range(rows):
        for px, py in ((x, 0), (x, cols - 1)):
            is_border = all(
                [
                    any([is_white(image[xx, py]) for xx in range(0, px)]), 
                    any([is_white(image[xx, py]) for xx in range(px+1, rows)]),
                ]
            )

            # print('px, py', px, py)
            if not is_border:
                bfs(px, py)
    for y in range(cols):
        for px, py in ((0, y), (rows - 1, y)):
            is_border = all(
                [
                    any([is_white(image[px, yy]) for yy in range(0, py)]), 
                    any([is_white(image[px, yy]) for yy in range(py+1, cols)]),
                ]
            )
            if not is_border:
                bfs(px, py)

    # Mark all non-visited (and non-white) pixels as red
    if label == 'RI':
        mark_color = [0, 0, 255]# Mark as blue
    elif label == 'RO':
        mark_color = [0, 255, 0]# Mark as green
    else:
        raise ValueError(f'No label: {label}')
        
    for i in range(rows):
        for j in range(cols):
            if not visited[i, j]:
                image[i, j] = mark_color

    return image

def process_image(image_path, output_path, label):
    image = load_image(image_path)
    result_image = mark_paths(image, label)
    result = Image.fromarray(result_image)
    result.save(output_path)  # Save the result as a PNG file

In [11]:
for _, row in tqdm(data_df.iterrows(), total=len(data_df)):
    data_path = row['data_path']
    gt_data_path = row['gt_data_path']
    output_path = data_path[:-4] + '_seg.png'
    label = os.path.basename(data_path).split('_')[1]
    process_image(gt_data_path, output_path, label)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4320/4320 [1:50:14<00:00,  1.53s/it]
